In [18]:
from pyspark.sql import SparkSession
# import pandas as pd

spark = SparkSession.builder.appName("uber-date-trips-sql").getOrCreate()

directory = "/home/steve/Capstone/IIS/Stock"
filename = "AAPL.csv"

data = spark.read.csv(f"file:///{directory}/{filename}", inferSchema = True, header = True)

# print(data)

In [14]:
from pyspark import SparkConf, SparkContext
import pandas as pd

conf = SparkConf().setMaster("local").setAppName("uber-date-trips")
sc = SparkContext(conf=conf)

directory = "/home/steve/Capstone/IIS/Stock"
filename = "GOOG.csv"

lines = sc.textFile(f"file:///{directory}/{filename}")

In [15]:
header = lines.first() 
filtered_lines = lines.filter(lambda row:row != header) 

In [16]:
filtered_lines.show(5)

AttributeError: 'PipelinedRDD' object has no attribute 'show'

In [19]:
data.show(5)

22/10/17 20:09:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Date, Open, High, Low, Close, Adj Close, Volume
 Schema: _c0, Date, Open, High, Low, Close, Adj Close, Volume
Expected: _c0 but found: 
CSV file: file:///home/steve/Capstone/IIS/Stock/AAPL.csv
+---+-------------------+---------+---------+---------+---------+---------+---------+
|_c0|               Date|     Open|     High|      Low|    Close|Adj Close|   Volume|
+---+-------------------+---------+---------+---------+---------+---------+---------+
|  0|2012-01-03 00:00:00|14.621429|14.732143|14.607143|14.686786|12.540045|302220800|
|  1|2012-01-04 00:00:00|14.642857|    14.81|14.617143|14.765714|12.607439|260022000|
|  2|2012-01-05 00:00:00|14.819643|14.948214|14.738214|14.929643|12.747405|271269600|
|  3|2012-01-06 00:00:00|14.991786|15.098214|14.972143|15.085714|12.880666|318292800|
|  4|2012-01-09 00:00:00|15.196429|15.276786|15.048214|15.061786|12.860234|394024400|
+---+-------------------

In [20]:
data.createOrReplaceTempView("APPLE_old_data")

In [21]:
spark.sql("select * from APPLE_old_data limit 5").show()

22/10/17 20:11:12 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Date, Open, High, Low, Close, Adj Close, Volume
 Schema: _c0, Date, Open, High, Low, Close, Adj Close, Volume
Expected: _c0 but found: 
CSV file: file:///home/steve/Capstone/IIS/Stock/AAPL.csv
+---+-------------------+---------+---------+---------+---------+---------+---------+
|_c0|               Date|     Open|     High|      Low|    Close|Adj Close|   Volume|
+---+-------------------+---------+---------+---------+---------+---------+---------+
|  0|2012-01-03 00:00:00|14.621429|14.732143|14.607143|14.686786|12.540045|302220800|
|  1|2012-01-04 00:00:00|14.642857|    14.81|14.617143|14.765714|12.607439|260022000|
|  2|2012-01-05 00:00:00|14.819643|14.948214|14.738214|14.929643|12.747405|271269600|
|  3|2012-01-06 00:00:00|14.991786|15.098214|14.972143|15.085714|12.880666|318292800|
|  4|2012-01-09 00:00:00|15.196429|15.276786|15.048214|15.061786|12.860234|394024400|
+---+-------------------

In [27]:
spark.sql("select Date, Open from APPLE_old_data").show()

+-------------------+---------+
|               Date|     Open|
+-------------------+---------+
|2012-01-03 00:00:00|14.621429|
|2012-01-04 00:00:00|14.642857|
|2012-01-05 00:00:00|14.819643|
|2012-01-06 00:00:00|14.991786|
|2012-01-09 00:00:00|15.196429|
|2012-01-10 00:00:00|15.211071|
|2012-01-11 00:00:00|15.095714|
|2012-01-12 00:00:00|15.081429|
|2012-01-13 00:00:00|14.989286|
|2012-01-17 00:00:00|    15.15|
|2012-01-18 00:00:00|15.248571|
|2012-01-19 00:00:00|  15.3625|
|2012-01-20 00:00:00|  15.2675|
|2012-01-23 00:00:00|15.095357|
|2012-01-24 00:00:00|15.182143|
|2012-01-25 00:00:00|    16.23|
|2012-01-26 00:00:00|16.012857|
|2012-01-27 00:00:00|15.869286|
|2012-01-30 00:00:00|15.918214|
|2012-01-31 00:00:00| 16.27107|
+-------------------+---------+
only showing top 20 rows



In [23]:
spark.sql("select Date, count(*) as open \
    from (select split(Date, ' ')[0] \
          as pickup_date from APPLE_old_data) \
          group by date ").explain(True)

AnalysisException: Column 'Date' does not exist. Did you mean one of the following? [__auto_generated_subquery_name.pickup_date]; line 1 pos 7;
'Aggregate ['date], ['Date, count(1) AS open#161L]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [split(cast(Date#51 as string),  , -1)[0] AS pickup_date#160]
      +- SubqueryAlias apple_old_data
         +- View (`APPLE_old_data`, [_c0#50,Date#51,Open#52,High#53,Low#54,Close#55,Adj Close#56,Volume#57])
            +- Relation [_c0#50,Date#51,Open#52,High#53,Low#54,Close#55,Adj Close#56,Volume#57] csv


In [17]:
sc.stop()